# Data Mover Notebook (Generalized Version)
## Prerequisites:
- Python 3.8
- PyCelonis 2.0 or later

In [2]:
# Install and import required packages
!python3.8 -m pip install --upgrade pip #run this command inside of your terminal
!pip install --upgrade pycelonis

from pathlib import Path #import Path class from pathlib module (part of standard library)
from pycelonis import get_celonis #import the get)celonis() method from pycelonis package
from pycelonis.pql import PQL, PQLColumn #PQL and PQLColumn classes imported from pycelonis.pql package path
from pycelonis.ems import ExportType, JobType
import os #library from core python

# Verify PyCelonis version
import pycelonis
print("PyCelonis version:", pycelonis.__version__)


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
ERROR: Could not find a version that satisfies the requirement pycelonis (from versions: none)
ERROR: No matching distribution found for pycelonis
You should consider upgrading via the '/Users/o.wu/venv/bin/python3 -m pip install --upgrade pip' command.
PyCelonis version: 2.11.0


## Step 1: Define Environment Variables
Replace the placeholders with your own details for the source and target environments.


In [6]:


import os
from pycelonis import get_celonis
import json
from pathlib import Path

# Load configuration from JSON file
config_path = Path("config.json")
if config_path.is_file():
    with open(config_path) as f:
        config = json.load(f)
else:
    raise FileNotFoundError("Configuration file 'config.json' not found in the current directory.")

# Fetch values from config file, throw error if any required field is missing
try:
    source_data_pool_id = config["source_data_pool_id"]
    source_data_model_id = config["source_data_model_id"]
    source_url = config["source_url"]
    source_api_token = config["source_api_token"]


    data_dir = Path(config.get("data_dir", Path().resolve() / "data"))
except KeyError as e:
    raise KeyError(f"Missing required configuration key: {e}")

print(f"Data directory: {data_dir}")

# Automatically create the data directory if it doesn't exist
data_dir.mkdir(parents=True, exist_ok=True)






Data directory: /Users/o.wu/Desktop/datamoverproject/data


## Step 2: Connect to Source and Target Environments


In [7]:
celonis_from = get_celonis(base_url = source_url, api_token = source_api_token, permissions=False, key_type='APP_KEY') #, key_type='USER_KEY'

datapool_from = celonis_from.data_integration.get_data_pool(source_data_pool_id)
datamodel_from = datapool_from.get_data_model(source_data_model_id)
datamodel_from_tables = datamodel_from.get_tables()

print("Tables in this datamodel:")
for tables in datamodel_from_tables:
    print(tables.name)   


Tables in this datamodel:
air_quality_no2


## Step 3: List Available Tables in Source Data Model


In [57]:
#import ipywidgets as widgets
#from IPython.display import display

# Button to trigger listing tables
list_tables_button = widgets.Button(description="List Tables in Source Data Model")
output_tables = widgets.Output()

def list_tables(b):
    with output_tables:
        output_tables.clear_output()  # Clear previous output
        
        # Retrieve and display tables in the source data model
        try:
            datamodel_from_tables = datamodel_from.get_tables()
            table_names = [table.name for table in datamodel_from_tables]

            # Display dropdown for table selection
            table_dropdown = widgets.Dropdown(
                options=table_names,
                description='Tables:',
                disabled=False,
            )
            
            print("Tables available in the source data model:")
            display(table_dropdown)
        except Exception as e:
            #throw error otherwise
            print(f"Error retrieving tables: {e}")

# Set up button action
list_tables_button.on_click(list_tables)

# Display the button and output area
display(list_tables_button, output_tables)





NameError: name 'widgets' is not defined

## Step 4: Specify Tables to Download from Source
Enter the names of the tables you want to download.


In [ ]:
# Specify the tables to download from the source
tables_to_download = ["Activity_Table_Demo", "Case_Table_Demo"]  # Update as needed
tables_from_detail = [datamodel_from_tables.find(table) for table in tables_to_download] #liste comprehension syntax

print("Tables to be downloaded:")
for table in tables_from_detail: #iterate through the array and print the name of each table 
    print(table.name) #print name attribute 


## Step 5: Download Data from Source Environment


In [2]:
# Function to download data from source and save it locally as Parquet files
def download_table_data(table):
    table_name = table.name
    query = PQL(distinct=False, limit=None)
    for column in table.get_columns():
        print(column)
        col_name = column.name
        if col_name != '_CELONIS_CHANGE_DATE':
            query += PQLColumn(name=col_name, query=f'"{table_name}"."{col_name}"')

    # Create export and download data as Parquet files
    data_export = datamodel_from.create_data_export(query=query, export_type=ExportType.PARQUET)
    data_export.wait_for_execution()

    for i, chunk in enumerate(data_export.get_chunks()): #index, object itself
        file_path = data_dir / f"{table_name}_{i}.parquet"
        with open(file_path, "wb") as file:
            file.write(chunk.read())
        print(f"Downloaded table: {table_name}, chunk: {i}")

for table in tables_from_detail:
    download_table_data(table)


NameError: name 'tables_from_detail' is not defined

## Step 6: Upload Data to Target Environment


In [ ]:
# List files to upload (ensure that only downloaded parquet files are included)
table_files = [f for f in os.listdir(data_dir) if f.endswith('.parquet')]

# Function to upload data to target environment
def upload_table_data(file_name):
    table_name = file_name.split(".")[0]
    data_push_job = datapool_to.create_data_push_job(target_name=table_name, type_=JobType.REPLACE)

    with open(data_dir / file_name, "rb") as file:
        data_push_job.add_file_chunk(file)

    data_push_job.execute(wait=True)
    print(f"Uploaded table: {table_name}")

for file_name in table_files:
    upload_table_data(file_name)


## Step 7: Add Uploaded Tables to Target Data Model (Optional)
If a target data model is specified, add the tables to it.


In [ ]:
if target_data_model_id: #if the target data model id exists and is not Null, then do the following
    datamodel_to = datapool_to.get_data_model(target_data_model_id)
    for file_name in table_files: #iterate over the table_files list
        table_name = file_name.split(".")[0]
        datamodel_to.add_table(name=table_name)
    datamodel_to.reload()
    print("Tables added to target data model and reloaded.")


## Notes
- **User Instructions**: Update variables in each section as per your environment’s requirements.
- **Modularity**: Functions for downloading and uploading tables improve reusability.
- **Logs and Feedback**: Each major step includes feedback, so the user knows the status of each operation.
